In [0]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   5129      0 --:--:-- --:--:-- --:--:--  5113
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 382.9MB 33kB/s 
+ set +ex
Installation succeeded!


In [0]:
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import struct
import os
import cupy as np
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy.ndimage import interpolation
import random
import sys
import numpy

## loading mnist dataset
import random

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                                % kind)
    images_path = os.path.join(path,
                               '%s-patterns-idx3-ubyte'
                               % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = numpy.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = numpy.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

#### Loading the data

X_train, y_train = load_mnist('./', kind='train')
X_test, y_test = load_mnist('./', kind='test')

# deskew

In [ ]:
def moments(image):
    c0,c1 = np.mgrid[:image.shape[0],:image.shape[1]] # A trick in numPy to create a mesh grid
    totalImage = np.sum(image) #sum of pixels
    m0 = np.sum(c0*image)/totalImage #mu_x
    m1 = np.sum(c1*image)/totalImage #mu_y
    m00 = np.sum((c0-m0)**2*image)/totalImage #var(x)
    m11 = np.sum((c1-m1)**2*image)/totalImage #var(y)
    m01 = np.sum((c0-m0)*(c1-m1)*image)/totalImage #covariance(x,y)
    mu_vector = np.array([m0,m1]) # Notice that these are \mu_x, \mu_y respectively
    covariance_matrix = np.array([[m00,m01],[m01,m11]]) # Do you see a similarity between the covariance matrix
    return mu_vector, covariance_matrix

In [ ]:
def deskew(image):
    c,v = moments(image)
    alpha = v[0,1]/v[0,0]
    affine = np.array([[1,0],[alpha,1]])
    ocenter = np.array(image.shape)/2.0
    offset = c-np.dot(affine,ocenter)
    return interpolation.affine_transform(image,affine,offset=offset)

In [ ]:
def deskewAll(X):
    currents = []
    for i in range(len(X)):
        currents.append(deskew(X[i].reshape(28,28)).flatten())
    return np.array(currents)

In [ ]:
X_train_deskewed = deskewAll(X_train)
X_test_deskewed = deskewAll(X_test)

In [0]:
X_train=X_train_deskewed/255
X_test=X_test_deskewed/255

# hog-72

In [ ]:
from skimage.feature import hog

def calc_hog_features(X, image_shape=(28, 28), pixels_per_cell=(8, 8)):
    fd_list = []
    for row in X:
        img = row.reshape(image_shape)
        fd = hog(img, orientations=8, pixels_per_cell=pixels_per_cell, cells_per_block=(1, 1))
        fd_list.append(fd)
    
    return np.array(fd_list)

In [ ]:
hoged_X_train = calc_hog_features(X_train, pixels_per_cell=(8, 8))
hoged_X_test = calc_hog_features(X_test, pixels_per_cell=(8, 8))
hoged_X_fea=hoged_X_train.shape[1]

# convolution-26*26

In [ ]:
kenel = [
    [1,0,-1],
    [1,0,-1],
    [1,0,-1]
]

In [ ]:
def convolution(X,kenel):
    con_X =np.zeros((26,26))
    for y in range(1,26):
        for x in range(1,26):
            temp = [
                [X[y*28+x-29],X[y*28+x-28],X[y*28+x-27]],
                [X[y*28+x-1],X[y*28+x],X[y*28+x+1]],
                [X[y*28+x+27],X[y*28+x+28],X[y*28+x+29]]
            ]
            mul_=np.array(temp)*cp.array(kenel)
            sum_=np.sum(mul_)
            con_X[y-1][x-1]=sum_
    return con_X

In [ ]:
con_X_train=convolution(X_train,kenel)
con_x_test=convolution(X_test,kenel)
con_X_fea=con_X_train.shape[1]

# classifier

In [88]:
class myClassifier(object):
    """
    ovr
    """
    def __init__(self, C=1000, eta=0.1, batch_size=20, epochs=100, epsilon=1e-8, 
                 shuffle=True, params=None, w=0, b=0):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.epsilon = epsilon
        self.class_num = 0
        self.shuffle = shuffle
        self.update_count = 0
        self.w = 0
        self.b = 0
        
    def fit(self, X, y, params=None, w=0, b=0, testscore = None, eval_score=None):
        # X_num = m, X_fea = n
        # m = np.shape(X)[0], n = np.shape(X)[1]
        
        #np# X_num, X_fea = np.shape(X)
        X_num = X.shape[0]
        X_fea = X.shape[1]
        #X_num=60000 X_fea=28*28
        self.class_num=len(np.unique(y))
        #class_num=10
        
        if params is None:
            print('fit params=None')
            self.params = {
                'w': np.random.randn(X_fea + con_X_fea + hoged_X_fea, self.class_num), #(10, 784) 정규분포난수
                'b': np.random.randn(1, self.class_num),
                'w_': np.random.randn(X_fea + con_X_fea + hoged_X_fea, self.class_num),
                'b_': np.random.randn(1, self.class_num),
                'tmpw': 0,
                'tmpb': 0
            }
        cnt=1
        if eval_score is None:
            self.score_val = 0
            
        for Xi in range(self.epochs):
            s_data, s_labels = self.shuffling(X, y) # tuple object
            encoded_y=self.encoding(s_labels)
            avg_loss = 0
            batch_count = int(X_num / self.batch_size)
            for t in range(int(batch_count)):
#                self.params['tmpw'] = temp_w, self.params['tmpb'] = temp_b
                batch_X, batch_y, bs = self.batching(s_data, encoded_y, t) # tuple:batch_X
                ###
                numpy.reshape(batch_X, (bs, X_fea + con_X_fea + hoged_X_fea))
                numpy.reshape(batch_y, (bs, self.class_num))
                ###
                print(type(batch_X))
                batch_X = np.array(batch_X)
                batch_y = np.array(batch_y)
                # batch_X = np.reshape(batch_X, (bs, X_fea))
                # batch_y = np.reshape(batch_y, (bs, self.class_num))
                z = self.net_input(batch_X)
                self.update_w_b(batch_X, batch_y, z, bs, cnt)
                cnt+=1
                self.update_count += 1

            self.params['tmpw'] = (cnt * (cnt/(cnt+1)) * 
                                   self.params['w_'] + (1/(cnt+1))*self.params['w'])
            self.params['tmpb'] = (cnt * (cnt/(cnt+1)) * 
                                   self.params['b_'] + (1/(cnt+1))*self.params['b'])
            prev_score = self.score_val
            pres_score = self.score(X, y)
            if Xi % 10 == 0:
                print("epochs: ", Xi)
                print("prev_score: ", prev_score)
                print("pres_score: ", pres_score)
                print()
            if prev_score < pres_score:
                self.score_val = pres_score
            if self.det_weight(X, y, 1) < self.det_weight(X, y): # temp_w, temp_b
                self.params['w_'] = self.params['tmpw']
                self.params['b_'] = self.params['tmpb']
        return self
    
    def det_weight(self, X, y, aver=0):
        if aver:
            w1 = self.params['w_']
            b1 = self.params['b_']
        else:
            w1 = self.params['tmpw']
            b1 = self.params['tmpb']
        temp = np.dot(X, w1) + b1
#         temp = temp.T
        pred = np.argmax(temp, axis=1)
        sco = np.mean(pred == y)
        return sco
    
    def update_w_b(self, batch_X, batch_y, z, bs, cnt):
        n = batch_X.shape[1]  # num of features
        delta_w = np.zeros(self.params['w'].shape)
        delta_b = np.zeros(self.params['b'].shape)
        z = np.reshape(z, (bs, self.class_num))
        temp = 1 - np.multiply(batch_y, z)
        temp[temp <= 0] = 0
        temp[temp > 0] = 1
        y_temp = np.multiply(batch_y, temp.reshape(bs, self.class_num))
        delta_w = -(1 / bs) * np.matmul(batch_X.T, y_temp) + (1 / self.C) * self.params['w']
        delta_b = -(1 / bs) * np.sum(y_temp, axis=0)
        self.params['w'] = self.params['w'] - (self.eta / (1 + self.epsilon * cnt)) * delta_w
        self.params['b'] = self.params['b'] - (self.eta / (1 + self.epsilon * cnt)) * delta_b
        
        return self.params
    
    def hinge_loss(self, y, z):
        loss = 1 - np.multiply(y, z)
        loss[loss < 0] = 0
        loss = np.mean(loss)
        return loss
    
    def net_input(self, X):  # net_input() = forward_prop(), generate z
        z = np.matmul(X, self.params['w']) + self.params['b']
        return z

    def encoding(self, y):
        y_array=np.array(y).reshape(-1,1)

        enc=OneHotEncoder()
        enc.fit(y_array)
        y_enc=enc.transform(y_array).toarray()
        
        encoded_y=-1*np.ones((np.shape(y_test)[0],10))
        
        y_enc=y_enc * 2 + encoded_y
        return y_enc

    def shuffling(self, X, y):
        temp_s=list(zip(X,y))
        random.shuffle(temp_s)
        X,y=zip(*temp_s)
        return X,y

    def batching(self, X, y, t):             
        batch_X = X[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        batch_y = y[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        last_size = min(len(X), (t+1) * self.batch_size) - t * self.batch_size
        
        return batch_X, batch_y, last_size
    
    def predict(self, X):
        m = X.shape[0]
        class_score = self.net_input(X)  # return z
        pred = np.argmax(class_score, axis=1)

        return pred
    
    def score(self, X, y):
        pred = self.predict(X)
        score = np.mean(pred == y)
        
        return score
    
    def get_params(self, deep=True):
        return {'C':self.C, 'batch_size':self.batch_size, 'epochs':self.epochs,
               'eta': self.eta, 'w':self.params['w'], 'b':self.params['b']}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def test(self, X, w, b):
        print("============== TESTING =================")
        z = np.dot(X, np.array(w)) + np.array(b)
        p = np.argmax(z, axis=1)
        return p

def main(training_image, training_label, test_image):
    ## loading mnist dataset ##
    raw_train = read_idx(training_image)
    train_data = np.reshape(raw_train, (X_train.shape[0], X_train.shape[1]))
    train_label = read_idx(training_label)

    raw_test = read_idx(test_image)
    test_data = np.reshape(raw_test, (X_test.shape[0], X_test.shape[1]))
    ## test_label = read_idx("./data/test-labels-idx1-ubyte")

    ## Standardzation ##
    # X_train_std = StandardScaler().fit_transform(train_data)
    # X_test_std = StandardScaler().fit_transform(test_data)
    X_train_std = train_data / 255
    X_test_std = test_data / 255

    ## SVM model ##
    mysvm = myClassifier(C=1000, batch_size=20, epochs=200, eta= 0.01).fit(X_train_std, train_label)

    w = mysvm.get_params()['w']
    b = mysvm.get_params()['b']

    pred = mysvm.test(X_test_std, w, b)

    file=open('./prediction.txt','w')
    for i in range(len(pred)):
        file.write('%s\n' %pred[i])
    file.close()

""" input
    argv : PATH
    eg) python team1.py ./data/newtrain-images-idx3-ubyte ./data/newtrain-labels-idx1-ubyte ./data/mnist_new_testall-patterns-idx3-ubyte
"""
# if __name__ == "__main__":
#     main(sys.argv[1], sys.argv[2], sys.argv[3])


' input\n    argv : PATH\n    eg) python team1.py ./data/newtrain-images-idx3-ubyte ./data/newtrain-labels-idx1-ubyte ./data/mnist_new_testall-patterns-idx3-ubyte\n'

In [0]:
x = np.array(X_train)
y = np.array(y_train)
# X_train_gpu = np.asarray(x)
# y_train_gpu = np.asarray(y)

In [90]:
mine=myClassifier()
mine.fit(x, y)

fit params=None
<class 'tuple'>


ValueError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')